In [6]:
!pip install nb_black lightgbm pyarrow umap-learn BorutaShap

In [2]:
%load_ext lab_black
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import glob
import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
import umap
import lightgbm as lgb
import numpy as np

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

from utils import DATA_SETS, gather_df, prepare_df

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [3]:
results = []

for dataset in DATA_SETS:
    df = gather_df(dataset, True)
    df = prepare_df(df, True, False, True)

    for target_col in [c for c in df.columns if "target" in c]:
        for shift in [30, 60]:
            y = df[target_col].pct_change(shift).shift(-shift)

            for split in range(3000, len(df.index) - 100, 100):

                X_train = df.iloc[:split]
                X_test = df.iloc[split + shift :]
                y_train = y.iloc[:split]
                y_test = y.iloc[split + shift :]

                x_filtered = ~pd.isna(y_train)

                if len(X_train[x_filtered]) < 1500:
                    continue
                if len(X_test) < 10:
                    continue
                y_train = y_train[x_filtered]

                rmodel = lgb.LGBMRegressor()
                rmodel.fit(X_train[x_filtered], np.clip(y_train, -1, 10))

                p = rmodel.predict(X_test)
                dfp = pd.DataFrame(
                    {
                        "p": p,
                        "y": y_test,
                        "y_orig": df[target_col].iloc[split + shift :] * y_test,
                        "p_orig": df[target_col].iloc[split + shift :] * p,
                    }
                ).iloc[:100]
                dfp = dfp[~pd.isna(dfp.y)]
                mae = np.mean(np.abs(dfp.p - dfp.y))
                rmse = np.sqrt(np.mean(dfp.p - dfp.y) ** 2)

                results.append(
                    {
                        "dataset": dataset,
                        "split": split,
                        "mae": mae,
                        "rmse": rmse,
                        "type": "all",
                        "target_col": target_col,
                        "shift": shift,
                        "mae_orig": np.mean(np.abs(dfp.p_orig - dfp.y_orig)),
                        "rmse_orig": np.sqrt(np.mean(dfp.p_orig - dfp.y_orig) ** 2),
                    }
                )

                pd.DataFrame(results).to_json("./simres-umap.json")